## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key(from the main folder)
#from config import g_key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Souillac,MU,-20.5167,57.5167,71.37,80,19,17.52,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,78.85,78,90,16.11,light rain
2,2,Linares,CL,-35.8500,-71.6000,58.86,70,63,2.44,broken clouds
3,3,Hermanus,ZA,-34.4187,19.2345,60.80,87,24,11.10,few clouds
4,4,Laguna,US,38.4210,-121.4238,98.20,34,1,5.01,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Souillac,MU,-20.5167,57.5167,71.37,80,19,17.52,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,78.85,78,90,16.11,light rain
5,5,Saint George,US,37.1041,-113.5841,94.78,17,1,1.01,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,78.30,76,75,19.86,broken clouds
14,14,Mahebourg,MU,-20.4081,57.7000,71.26,78,29,18.34,scattered clouds
15,15,Mandera,KE,3.9366,41.8670,81.19,57,91,16.89,overcast clouds
16,16,Sao Filipe,CV,14.8961,-24.4956,78.66,76,36,4.34,scattered clouds
17,17,Pauini,BR,-7.7136,-66.9764,84.69,59,86,0.56,overcast clouds
20,20,Albany,US,42.6001,-73.9662,69.53,71,93,2.82,overcast clouds
24,24,Rikitea,PF,-23.1203,-134.9692,73.90,76,58,14.74,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                371
City                   371
Country                368
Lat                    371
Lng                    371
Max Temp               371
Humidity               371
Cloudiness             371
Wind Speed             371
Current Description    371
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Souillac,MU,-20.5167,57.5167,71.37,80,19,17.52,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,78.85,78,90,16.11,light rain
5,5,Saint George,US,37.1041,-113.5841,94.78,17,1,1.01,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,78.30,76,75,19.86,broken clouds
14,14,Mahebourg,MU,-20.4081,57.7000,71.26,78,29,18.34,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
670,670,Bandarbeyla,SO,9.4942,50.8122,77.14,86,49,19.77,scattered clouds
673,673,Belen,TR,36.4891,36.2233,65.03,72,0,2.30,clear sky
675,675,Bolobo,CD,-2.1667,16.2333,74.68,73,63,2.30,broken clouds
676,676,Qiongshan,CN,20.0058,110.3542,75.18,94,0,4.47,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Souillac,MU,71.37,few clouds,-20.5167,57.5167,
1,Avarua,CK,78.85,light rain,-21.2078,-159.7750,
5,Saint George,US,94.78,clear sky,37.1041,-113.5841,
7,Atuona,PF,78.30,broken clouds,-9.8000,-139.0333,
14,Mahebourg,MU,71.26,scattered clouds,-20.4081,57.7000,
15,Mandera,KE,81.19,overcast clouds,3.9366,41.8670,
16,Sao Filipe,CV,78.66,scattered clouds,14.8961,-24.4956,
17,Pauini,BR,84.69,overcast clouds,-7.7136,-66.9764,
20,Albany,US,69.53,overcast clouds,42.6001,-73.9662,
24,Rikitea,PF,73.90,light rain,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.notnull().sum()

City                   368
Country                368
Max Temp               368
Current Description    368
Lat                    368
Lng                    368
Hotel Name             368
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Souillac,MU,71.37,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
1,Avarua,CK,78.85,light rain,-21.2078,-159.7750,Paradise Inn
5,Saint George,US,94.78,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
7,Atuona,PF,78.30,broken clouds,-9.8000,-139.0333,Villa Enata
14,Mahebourg,MU,71.26,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# # Save the figure.
#savemap("WeatherPy_vacation_map.png")

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))